# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1:
*   Alumno 2:
*   Alumno 3:






---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [35]:
# # ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
# mount='/content/gdrive'
# drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

# try:
#   from google.colab import drive
#   IN_COLAB=True
# except:
#   IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [36]:
# # Switch to the directory on the Google Drive that you want to use
# import os
# if IN_COLAB:
#   print("We're running Colab")

#   if IN_COLAB:
#     # Mount the Google Drive at mount
#     print("Colab: mounting Google drive on ", mount)

#     drive.mount(mount)

#     # Create drive_root if it doesn't exist
#     create_drive_root = True
#     if create_drive_root:
#       print("\nColab: making sure ", drive_root, " exists.")
#       os.makedirs(drive_root, exist_ok=True)

#     # Change to the directory
#     print("\nColab: Changing directory to ", drive_root)
#     %cd $drive_root
# # Verify we're in the correct working directory
# %pwd
# print("Archivos en el directorio: ")
# print(os.listdir())

---
### 1.4. Instalar librerías necesarias

In [37]:
# if IN_COLAB:
#   %pip install gym==0.17.3
#   %pip install git+https://github.com/Kojoley/atari-py.git
#   %pip install keras-rl2==1.0.5
#   %pip install tensorflow==2.8
# else:
#   %pip install gym==0.17.3
#   %pip install git+https://github.com/Kojoley/atari-py.git
#   %pip install pyglet==1.5.0
#   %pip install h5py==3.1.0
#   %pip install Pillow==9.5.0
#   %pip install keras-rl2==1.0.5
#   %pip install Keras==2.2.4
#   %pip install tensorflow==2.5.3
#   %pip install torch==2.0.1
#   %pip install agents==1.4.0

---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [1]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Dense, Activation, Flatten, Convolution2D, Permute, Input,
                                     SeparableConv2D, Lambda, Subtract, Add, Conv2D, 
                                     DepthwiseConv2D, BatchNormalization, ReLU, InputLayer)
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

import os
import tensorflow as tf
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K 
import time
from tensorflow.keras.callbacks import Callback
from rl.callbacks import FileLogger


In [ ]:
path = 'POC2'

# Define subdirectorios
videos_path = os.path.join(path, 'videos')
modelos_path = os.path.join(path, 'modelos')

# Crear directorios si no existen
os.makedirs(videos_path, exist_ok=True)
os.makedirs(modelos_path, exist_ok=True)

#### Configuración base

In [2]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [3]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

In [4]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


1. Implementación de la red neuronal

In [5]:
# Hiperparámetros

LEARNING_RATE = 0.0005

GAMMA = 0.99
TARGET_MODEL_UPDATE = 10000
DELTA_CLIP = 1.0
TRAIN_INTERVAL = 24

Vamos a comparar con diferentes arquitecturas de red para ver cual podría darnos mejores resultados

##### Modelo 1

In [6]:
def create_dqn_model(input_shape, num_actions):
    model = Sequential()
    model.add(InputLayer(input_shape=(input_shape[0], input_shape[1], input_shape[2]))) #(84, 84, 1)
    model.add(Permute((2, 3, 1)))
    model.add(Conv2D(32, (8, 8), strides=4, activation='relu'))
    model.add(Conv2D(64, (4, 4), strides=2, activation='relu'))
    model.add(Conv2D(64, (3, 3), strides=1, activation='relu'))
    # Flattening layer
    model.add(Flatten())
    # Fully connected (Dense) layers
    model.add(Dense(512, activation='relu'))
    model.add(Dense(num_actions, activation='linear'))
    
    return model

# Prueba rápida
model = create_dqn_model((4, 84, 84), num_actions=6)
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 20, 20, 32)        8224      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1606144   
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3

##### Modelo 2

In [7]:
def create_separable_light_dqn(input_shape, num_actions):
    x_in = Input(shape=(input_shape[0], input_shape[1], input_shape[2]))
    x = Permute((2, 3, 1))(x_in)
    x = SeparableConv2D(16, (8, 8), strides=4, activation='relu', padding='same')(x)
    x = SeparableConv2D(32, (4, 4), strides=2, activation='relu', padding='same')(x)
    x = SeparableConv2D(32, (3, 3), strides=1, activation='relu', padding='same')(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(num_actions, activation='linear', dtype='float32')(x)
    return Model(inputs=x_in, outputs=x)

# Prueba rápida
model = create_separable_light_dqn((4, 84, 84), num_actions=6)
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 4, 84, 84)]       0         
_________________________________________________________________
permute_1 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
separable_conv2d (SeparableC (None, 21, 21, 16)        336       
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 11, 11, 32)        800       
_________________________________________________________________
separable_conv2d_2 (Separabl (None, 11, 11, 32)        1344      
_________________________________________________________________
flatten_1 (Flatten)          (None, 3872)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               991488

##### Modelo 3

In [8]:
def create_dueling_light_dqn(input_shape, num_actions):
    x_in = Input(shape=(input_shape[0], input_shape[1], input_shape[2]))  # (C, H, W)
    
    # Reordenar a formato channels_last (H, W, C)
    x = Permute((2, 3, 1))(x_in)
    
    # Convoluciones separables
    x = SeparableConv2D(16, (8, 8), strides=4, activation='relu', padding='same')(x)
    x = SeparableConv2D(32, (4, 4), strides=2, activation='relu', padding='same')(x)
    x = SeparableConv2D(32, (3, 3), strides=1, activation='relu', padding='same')(x)
    x = Flatten()(x)

    # Stream ventaja (Advantage)
    adv = Dense(128, activation='relu')(x)
    adv = Dense(num_actions, activation='linear')(adv)

    # Stream valor (Value)
    val = Dense(128, activation='relu')(x)
    val = Dense(1, activation='linear')(val)

    # Dueling: Q(s,a) = V(s) + (A(s,a) - mean(A(s,·)))
    adv_mean = Lambda(lambda a: K.mean(a, axis=1, keepdims=True))(adv)
    adv_centered = Subtract()([adv, adv_mean])
    q = Add()([val, adv_centered])

    return Model(inputs=x_in, outputs=q)

# Prueba rápida
model = create_dueling_light_dqn((4, 84, 84), num_actions=6)
model.compile(optimizer='adam', loss='mse')
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 4, 84, 84)]  0                                            
__________________________________________________________________________________________________
permute_2 (Permute)             (None, 84, 84, 4)    0           input_3[0][0]                    
__________________________________________________________________________________________________
separable_conv2d_3 (SeparableCo (None, 21, 21, 16)   336         permute_2[0][0]                  
__________________________________________________________________________________________________
separable_conv2d_4 (SeparableCo (None, 11, 11, 32)   800         separable_conv2d_3[0][0]         
____________________________________________________________________________________________

##### Modelo 4

In [9]:
def create_mobile_like_dqn(input_shape, num_actions):
    x_in = Input(shape=(input_shape[0], input_shape[1], input_shape[2]))
    x = Permute((2, 3, 1))(x_in)  # (84,84,frames)
    # Primer conv estándar ligero
    x = Conv2D(16, (3, 3), strides=2, padding='same')(x)  # ~(42x42x16)
    x = BatchNormalization()(x); x = ReLU()(x)
    # Bloque depthwise-pointwise
    def dw_pw(block_input, filters, strides):
        y = DepthwiseConv2D((3,3), strides=strides, padding='same')(block_input)
        y = BatchNormalization()(y); y = ReLU()(y)
        y = Conv2D(filters, (1,1), padding='same')(y)
        y = BatchNormalization()(y); y = ReLU()(y)
        return y
    x = dw_pw(x, 32, strides=2)   # ~(21x21x32)
    x = dw_pw(x, 32, strides=2)   # ~(11x11x32)
    x = dw_pw(x, 64, strides=1)   # ~(11x11x64)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(num_actions, activation='linear', dtype='float32')(x)
    return Model(inputs=x_in, outputs=x)

# Prueba rápida
model = create_mobile_like_dqn((4, 84, 84), num_actions=6)
model.compile(optimizer='adam', loss='mse')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 4, 84, 84)]       0         
_________________________________________________________________
permute_3 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 42, 42, 16)        592       
_________________________________________________________________
batch_normalization (BatchNo (None, 42, 42, 16)        64        
_________________________________________________________________
re_lu (ReLU)                 (None, 42, 42, 16)        0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 21, 21, 16)        160       
_____________________________________________________

In [10]:
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
num_actions = nb_actions

model = create_dqn_model(input_shape, num_actions)
model_separable_light_dqn = create_separable_light_dqn(input_shape, num_actions)
model_dueling_light_dqn = create_dueling_light_dqn(input_shape, num_actions)
model_like_dqn = create_mobile_like_dqn(input_shape, num_actions)


2. Implementación de la solución DQN

In [11]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1.0, value_min=0.1, value_test=0.05, nb_steps=1000000)


Callbacks

In [12]:
# Definimos un Callbak para poder guardar los mejores pesos cada 20 iteraciones 
class SaveBestWeights(Callback):
    def __init__(self, filepath, interval=20, overwrite=True):
        super(SaveBestWeights, self).__init__()
        self.filepath = filepath
        self.interval = interval
        self.best_reward = -float('inf')
        self.overwrite = overwrite
    
    def on_episode_end(self, episode, logs={}):
        if episode % self.interval == 0:
            current_reward = logs.get('episode_reward')
            if current_reward > self.best_reward:
                self.best_reward = current_reward
                if self.overwrite or not os.path.exists(self.filepath):
                    self.model.save_weights(self.filepath, overwrite=True)
                    print(f'\n ##### Pesos guardados en {self.filepath} recompensa: {self.best_reward}')
                else:
                    print(f'\n ##### Warning: No se sobrescribió {self.filepath}, ya existe.')


# Definimos un Callbak para poder guardar los ultimos pesos cada 10 iteraciones, sin importar si mejoró el resultado del modelo 
class SaveLastWeights(Callback):
    def __init__(self, filepath, interval=10):
        super(SaveLastWeights, self).__init__()
        self.filepath = filepath
        self.interval = interval
    
    def on_episode_end(self, episode, logs={}):
        if episode % self.interval == 0:
            self.model.save_weights(self.filepath, overwrite=True)
            print(f'\n ##### Pesos guardados en {self.filepath} al final del episodio {episode}')

In [13]:
filepath_bw = os.path.join(modelos_path, 'dqn_best_weights.h5')
save_best_weights = SaveBestWeights(filepath=filepath_bw, interval=50, overwrite=True)

filepath_lw = os.path.join(modelos_path, 'dqn_last_weights.h5')
save_last_weights = SaveLastWeights(filepath=filepath_lw, interval=50)

NameError: name 'modelos_path' is not defined

Agentes

In [14]:
dqn = DQNAgent(
    model=model,
    nb_actions=num_actions,
    policy=policy,
    memory=memory,
    processor=processor,
    nb_steps_warmup = 50000, # indica cuántos pasos (interacciones con el entorno) el agente debe ejecutar antes de comenzar a entrenar el modelo
    gamma=GAMMA,
    target_model_update = TARGET_MODEL_UPDATE,
    train_interval = TRAIN_INTERVAL,
    delta_clip = DELTA_CLIP
)

optimizer = Adam(learning_rate = LEARNING_RATE)
dqn.compile(optimizer=optimizer, metrics=['mae'])

In [15]:
dqn_separable_light_dqn = DQNAgent(
    model=model_separable_light_dqn,
    nb_actions=num_actions,
    policy=policy,
    memory=memory,
    processor=processor,
    nb_steps_warmup = 50000, # indica cuántos pasos (interacciones con el entorno) el agente debe ejecutar antes de comenzar a entrenar el modelo
    gamma=GAMMA,
    target_model_update = TARGET_MODEL_UPDATE,
    train_interval = TRAIN_INTERVAL,
    delta_clip = DELTA_CLIP
)

optimizer = Adam(learning_rate = LEARNING_RATE)
dqn_separable_light_dqn.compile(optimizer=optimizer, metrics=['mae'])

In [16]:
dqn_dueling_light_dqn = DQNAgent(
    model=model_dueling_light_dqn,
    nb_actions=num_actions,
    policy=policy,
    memory=memory,
    processor=processor,
    nb_steps_warmup = 50000, # indica cuántos pasos (interacciones con el entorno) el agente debe ejecutar antes de comenzar a entrenar el modelo
    gamma=GAMMA,
    target_model_update = TARGET_MODEL_UPDATE,
    train_interval = TRAIN_INTERVAL,
    delta_clip = DELTA_CLIP
)

optimizer = Adam(learning_rate = LEARNING_RATE)
dqn_dueling_light_dqn.compile(optimizer=optimizer, metrics=['mae'])

d:\anaconda3\envs\gym4\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [17]:
dqn_like_dqn = DQNAgent(
    model=model_like_dqn,
    nb_actions=num_actions,
    policy=policy,
    memory=memory,
    processor=processor,
    nb_steps_warmup = 50000, # indica cuántos pasos (interacciones con el entorno) el agente debe ejecutar antes de comenzar a entrenar el modelo
    gamma=GAMMA,
    target_model_update = TARGET_MODEL_UPDATE,
    train_interval = TRAIN_INTERVAL,
    delta_clip = DELTA_CLIP
)

optimizer = Adam(learning_rate = LEARNING_RATE)
dqn_like_dqn.compile(optimizer=optimizer, metrics=['mae'])

In [18]:
log_filename = os.path.join(modelos_path, f'dqn_{env_name}_log.json')

NameError: name 'modelos_path' is not defined

#### Prueba con numero de iteraciones = 100.000

In [ ]:
path = 'POC1'

# Define subdirectorios
videos_path = os.path.join(path, 'videos')
modelos_path = os.path.join(path, 'modelos')

# Crear directorios si no existen
os.makedirs(videos_path, exist_ok=True)
os.makedirs(modelos_path, exist_ok=True)

log_filename = os.path.join(modelos_path, f'dqn_{env_name}_log.json')

filepath_bw = os.path.join(modelos_path, 'dqn_best_weights.h5')
save_best_weights = SaveBestWeights(filepath=filepath_bw, interval=50, overwrite=True)

filepath_lw = os.path.join(modelos_path, 'dqn_last_weights.h5')
save_last_weights = SaveLastWeights(filepath=filepath_lw, interval=50)

print("########--------------  modelo 1  ------------------###########")

start_time = time.time()

train_scores = dqn.fit(env, nb_steps=100000, visualize=False, verbose=1, log_interval=5000, 
                       callbacks=[save_best_weights, save_last_weights, FileLogger(log_filename, interval=100)])

end_time = time.time()
training_duration = end_time - start_time

print(f"Entrenamiento completado en {training_duration:.2f} segundos ({training_duration/60:.2f} minutos)")

weights_filename = os.path.join(modelos_path, 'dqn_last_weights.h5')
dqn.load_weights(weights_filename)
test_scores = dqn.test(env, nb_episodes=10, visualize=False)

episode_rewards = test_scores.history['episode_reward']
print(int(np.min(episode_rewards)), "~", int(np.max(episode_rewards)), "Mean", np.mean(episode_rewards))

########--------------  modelo 1  ------------------###########
Training for 100000 steps ...
Interval 1 (0 steps performed)
1198/5000 [======>.......................] - ETA: 24s - reward: 0.0042Pesos guardados en POC\modelos\dqn_best_weights.h5 recompensa: 5.0
Pesos guardados en POC\modelos\dqn_last_weights.h5 al final del episodio 0
5000/5000 [==============================] - 36s 6ms/step - reward: 0.0080
4 episodes - episode_reward: 9.250 [5.000, 13.000] - ale.lives: 2.117

Interval 2 (5000 steps performed)
5000/5000 [==============================] - 29s 6ms/step - reward: 0.0060
6 episodes - episode_reward: 5.000 [3.000, 8.000] - ale.lives: 2.090

Interval 3 (10000 steps performed)
5000/5000 [==============================] - 31s 6ms/step - reward: 0.0084
5 episodes - episode_reward: 8.200 [4.000, 12.000] - ale.lives: 2.440

Interval 4 (15000 steps performed)
5000/5000 [==============================] - 32s 6ms/step - reward: 0.0068
6 episodes - episode_reward: 6.000 [3.000, 14.0

In [58]:
path = 'POC2'

# Define subdirectorios
videos_path = os.path.join(path, 'videos')
modelos_path = os.path.join(path, 'modelos')

# Crear directorios si no existen
os.makedirs(videos_path, exist_ok=True)
os.makedirs(modelos_path, exist_ok=True)

log_filename = os.path.join(modelos_path, f'dqn_{env_name}_log.json')

filepath_bw = os.path.join(modelos_path, 'dqn_best_weights.h5')
save_best_weights = SaveBestWeights(filepath=filepath_bw, interval=50, overwrite=True)

filepath_lw = os.path.join(modelos_path, 'dqn_last_weights.h5')
save_last_weights = SaveLastWeights(filepath=filepath_lw, interval=50)

print("########--------------  modelo 2  ------------------###########")

start_time = time.time()

train_scores = dqn_separable_light_dqn.fit(env, nb_steps=100000, visualize=False, verbose=1, log_interval=5000, 
                       callbacks=[save_best_weights, save_last_weights, FileLogger(log_filename, interval=100)])

end_time = time.time()
training_duration = end_time - start_time

print(f"Entrenamiento completado en {training_duration:.2f} segundos ({training_duration/60:.2f} minutos)")

weights_filename = os.path.join(modelos_path, 'dqn_last_weights.h5')
dqn_separable_light_dqn.load_weights(weights_filename)
test_scores = dqn_separable_light_dqn.test(env, nb_episodes=10, visualize=False)

episode_rewards = test_scores.history['episode_reward']
print(int(np.min(episode_rewards)), "~", int(np.max(episode_rewards)), "Mean", np.mean(episode_rewards))

########--------------  modelo 2  ------------------###########
Training for 100000 steps ...
Interval 1 (0 steps performed)
 883/5000 [====>.........................] - ETA: 24s - reward: 0.0147Pesos guardados en POC2\modelos\dqn_best_weights.h5 recompensa: 13.0
Pesos guardados en POC2\modelos\dqn_last_weights.h5 al final del episodio 0
5000/5000 [==============================] - 41s 7ms/step - reward: 0.0130
5 episodes - episode_reward: 11.200 [6.000, 14.000] - ale.lives: 2.298

Interval 2 (5000 steps performed)
5000/5000 [==============================] - 31s 6ms/step - reward: 0.0120
7 episodes - episode_reward: 9.857 [5.000, 16.000] - ale.lives: 2.099

Interval 3 (10000 steps performed)
5000/5000 [==============================] - 31s 6ms/step - reward: 0.0128
7 episodes - episode_reward: 8.429 [4.000, 12.000] - ale.lives: 2.005

Interval 4 (15000 steps performed)
5000/5000 [==============================] - 30s 6ms/step - reward: 0.0108
6 episodes - episode_reward: 6.833 [5.000,

In [59]:
path = 'POC3'

# Define subdirectorios
videos_path = os.path.join(path, 'videos')
modelos_path = os.path.join(path, 'modelos')

# Crear directorios si no existen
os.makedirs(videos_path, exist_ok=True)
os.makedirs(modelos_path, exist_ok=True)

log_filename = os.path.join(modelos_path, f'dqn_{env_name}_log.json')

filepath_bw = os.path.join(modelos_path, 'dqn_best_weights.h5')
save_best_weights = SaveBestWeights(filepath=filepath_bw, interval=50, overwrite=True)

filepath_lw = os.path.join(modelos_path, 'dqn_last_weights.h5')
save_last_weights = SaveLastWeights(filepath=filepath_lw, interval=50)

print("########--------------  modelo 3  ------------------###########")

start_time = time.time()

train_scores = dqn_dueling_light_dqn.fit(env, nb_steps=100000, visualize=False, verbose=1, log_interval=5000, 
                       callbacks=[save_best_weights, save_last_weights, FileLogger(log_filename, interval=100)])

end_time = time.time()
training_duration = end_time - start_time

print(f"Entrenamiento completado en {training_duration:.2f} segundos ({training_duration/60:.2f} minutos)")

weights_filename = os.path.join(modelos_path, 'dqn_last_weights.h5')
dqn_dueling_light_dqn.load_weights(weights_filename)
test_scores = dqn_dueling_light_dqn.test(env, nb_episodes=10, visualize=False)

episode_rewards = test_scores.history['episode_reward']
print(int(np.min(episode_rewards)), "~", int(np.max(episode_rewards)), "Mean", np.mean(episode_rewards))

########--------------  modelo 3  ------------------###########
Training for 100000 steps ...
Interval 1 (0 steps performed)
1168/5000 [======>.......................] - ETA: 23s - reward: 0.0137Pesos guardados en POC3\modelos\dqn_best_weights.h5 recompensa: 16.0
Pesos guardados en POC3\modelos\dqn_last_weights.h5 al final del episodio 0
5000/5000 [==============================] - 42s 7ms/step - reward: 0.0142
5 episodes - episode_reward: 14.000 [9.000, 16.000] - ale.lives: 2.127

Interval 2 (5000 steps performed)
5000/5000 [==============================] - 30s 6ms/step - reward: 0.0162
4 episodes - episode_reward: 17.000 [10.000, 23.000] - ale.lives: 2.401

Interval 3 (10000 steps performed)
5000/5000 [==============================] - 33s 7ms/step - reward: 0.0156
6 episodes - episode_reward: 15.333 [11.000, 24.000] - ale.lives: 2.185

Interval 4 (15000 steps performed)
5000/5000 [==============================] - 31s 6ms/step - reward: 0.0148
4 episodes - episode_reward: 15.500 [8

In [60]:
path = 'POC4'

# Define subdirectorios
videos_path = os.path.join(path, 'videos')
modelos_path = os.path.join(path, 'modelos')

# Crear directorios si no existen
os.makedirs(videos_path, exist_ok=True)
os.makedirs(modelos_path, exist_ok=True)

log_filename = os.path.join(modelos_path, f'dqn_{env_name}_log.json')

filepath_bw = os.path.join(modelos_path, 'dqn_best_weights.h5')
save_best_weights = SaveBestWeights(filepath=filepath_bw, interval=50, overwrite=True)

filepath_lw = os.path.join(modelos_path, 'dqn_last_weights.h5')
save_last_weights = SaveLastWeights(filepath=filepath_lw, interval=50)

print("########--------------  modelo 4  ------------------###########")

start_time = time.time()

train_scores = dqn_like_dqn.fit(env, nb_steps=100000, visualize=False, verbose=1, log_interval=5000, 
                       callbacks=[save_best_weights, save_last_weights, FileLogger(log_filename, interval=100)])

end_time = time.time()
training_duration = end_time - start_time

print(f"Entrenamiento completado en {training_duration:.2f} segundos ({training_duration/60:.2f} minutos)")

weights_filename = os.path.join(modelos_path, 'dqn_last_weights.h5')
dqn_like_dqn.load_weights(weights_filename)
test_scores = dqn_like_dqn.test(env, nb_episodes=10, visualize=False)

episode_rewards = test_scores.history['episode_reward']
print(int(np.min(episode_rewards)), "~", int(np.max(episode_rewards)), "Mean", np.mean(episode_rewards))

########--------------  modelo 4  ------------------###########
Training for 100000 steps ...
Interval 1 (0 steps performed)
 596/5000 [==>...........................] - ETA: 32s - reward: 0.0067Pesos guardados en POC4\modelos\dqn_best_weights.h5 recompensa: 4.0
Pesos guardados en POC4\modelos\dqn_last_weights.h5 al final del episodio 0
5000/5000 [==============================] - 48s 9ms/step - reward: 0.0124
8 episodes - episode_reward: 7.750 [3.000, 14.000] - ale.lives: 2.116

Interval 2 (5000 steps performed)
5000/5000 [==============================] - 38s 8ms/step - reward: 0.0130
9 episodes - episode_reward: 7.222 [4.000, 13.000] - ale.lives: 2.178

Interval 3 (10000 steps performed)
5000/5000 [==============================] - 38s 8ms/step - reward: 0.0146
7 episodes - episode_reward: 10.286 [4.000, 19.000] - ale.lives: 2.136

Interval 4 (15000 steps performed)
5000/5000 [==============================] - 38s 8ms/step - reward: 0.0142
8 episodes - episode_reward: 7.500 [5.000, 

Resumen de lo obtenido para número de iteraciones = 100.000 


| Model ID | Nombre del Modelo | Recompensas Totales | Recompensa Media | Duración prueba |
|----------|--------------------|----------------------|-------------------|-------------------|
| Model 1  | create_dqn_model         | 11 ~ 28                | 18.6               | 14 m 40 seg 
| Model 2  | create_separable_light_dqn       | 12 ~ 26                | 18.3               | 17 m 58 seg
| Model 3  | create_dueling_light_dqn | 18 ~ 18     | 18               | 17 m
| Model 4  | create_mobile_like_dqn | 18 ~ 20 | 19.6 | 26 m 11 seg

#### Prueba con numero de iteraciones = 250.000

In [62]:
path = 'POC1'

# Define subdirectorios
videos_path = os.path.join(path, 'videos')
modelos_path = os.path.join(path, 'modelos')

# Crear directorios si no existen
os.makedirs(videos_path, exist_ok=True)
os.makedirs(modelos_path, exist_ok=True)

log_filename = os.path.join(modelos_path, f'dqn_{env_name}_log.json')

filepath_bw = os.path.join(modelos_path, 'dqn_best_weights.h5')
save_best_weights = SaveBestWeights(filepath=filepath_bw, interval=50, overwrite=True)

filepath_lw = os.path.join(modelos_path, 'dqn_last_weights.h5')
save_last_weights = SaveLastWeights(filepath=filepath_lw, interval=50)

print("########--------------  modelo 1  ------------------###########")

start_time = time.time()

train_scores = dqn.fit(env, nb_steps=250000, visualize=False, verbose=1, log_interval=10000, 
                       callbacks=[save_best_weights, save_last_weights, FileLogger(log_filename, interval=100)])

end_time = time.time()
training_duration = end_time - start_time

print(f"Entrenamiento completado en {training_duration:.2f} segundos ({training_duration/60:.2f} minutos)")

weights_filename = os.path.join(modelos_path, 'dqn_last_weights.h5')
dqn.load_weights(weights_filename)
test_scores = dqn.test(env, nb_episodes=10, visualize=False)

episode_rewards = test_scores.history['episode_reward']
print(int(np.min(episode_rewards)), "~", int(np.max(episode_rewards)), "Mean", np.mean(episode_rewards))

########--------------  modelo 1  ------------------###########
Training for 250000 steps ...
Interval 1 (0 steps performed)
  364/10000 [>.............................] - ETA: 1:59 - reward: 0.0165Pesos guardados en POC1\modelos\dqn_best_weights.h5 recompensa: 6.0
Pesos guardados en POC1\modelos\dqn_last_weights.h5 al final del episodio 0
10000/10000 [==============================] - 130s 8ms/step - reward: 0.0222
10 episodes - episode_reward: 21.500 [6.000, 35.000] - ale.lives: 2.234

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 89s 9ms/step - reward: 0.0213
10 episodes - episode_reward: 20.800 [9.000, 34.000] - ale.lives: 2.268

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 65s 6ms/step - reward: 0.0222
11 episodes - episode_reward: 21.091 [16.000, 31.000] - ale.lives: 2.196

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 65s 6ms/step - reward: 0.0219
12 episodes - episode_re

In [63]:
path = 'POC2'

# Define subdirectorios
videos_path = os.path.join(path, 'videos')
modelos_path = os.path.join(path, 'modelos')

# Crear directorios si no existen
os.makedirs(videos_path, exist_ok=True)
os.makedirs(modelos_path, exist_ok=True)

log_filename = os.path.join(modelos_path, f'dqn_{env_name}_log.json')

filepath_bw = os.path.join(modelos_path, 'dqn_best_weights.h5')
save_best_weights = SaveBestWeights(filepath=filepath_bw, interval=50, overwrite=True)

filepath_lw = os.path.join(modelos_path, 'dqn_last_weights.h5')
save_last_weights = SaveLastWeights(filepath=filepath_lw, interval=50)

print("########--------------  modelo 2  ------------------###########")

start_time = time.time()

train_scores = dqn_separable_light_dqn.fit(env, nb_steps=250000, visualize=False, verbose=1, log_interval=10000, 
                       callbacks=[save_best_weights, save_last_weights, FileLogger(log_filename, interval=100)])

end_time = time.time()
training_duration = end_time - start_time

print(f"Entrenamiento completado en {training_duration:.2f} segundos ({training_duration/60:.2f} minutos)")

weights_filename = os.path.join(modelos_path, 'dqn_last_weights.h5')
dqn_separable_light_dqn.load_weights(weights_filename)
test_scores = dqn_separable_light_dqn.test(env, nb_episodes=10, visualize=False)

episode_rewards = test_scores.history['episode_reward']
print(int(np.min(episode_rewards)), "~", int(np.max(episode_rewards)), "Mean", np.mean(episode_rewards))

########--------------  modelo 2  ------------------###########
Training for 250000 steps ...
Interval 1 (0 steps performed)
 1122/10000 [==>...........................] - ETA: 1:01 - reward: 0.0152Pesos guardados en POC2\modelos\dqn_best_weights.h5 recompensa: 17.0
Pesos guardados en POC2\modelos\dqn_last_weights.h5 al final del episodio 0
10000/10000 [==============================] - 74s 7ms/step - reward: 0.0179
9 episodes - episode_reward: 18.111 [14.000, 24.000] - ale.lives: 1.746

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 67s 7ms/step - reward: 0.0208
11 episodes - episode_reward: 19.273 [12.000, 27.000] - ale.lives: 1.981

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 67s 7ms/step - reward: 0.0198
12 episodes - episode_reward: 16.167 [10.000, 21.000] - ale.lives: 2.099

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 67s 7ms/step - reward: 0.0193
11 episodes - episode_r

In [19]:
path = 'POC3'

# Define subdirectorios
videos_path = os.path.join(path, 'videos')
modelos_path = os.path.join(path, 'modelos')

# Crear directorios si no existen
os.makedirs(videos_path, exist_ok=True)
os.makedirs(modelos_path, exist_ok=True)

log_filename = os.path.join(modelos_path, f'dqn_{env_name}_log.json')

filepath_bw = os.path.join(modelos_path, 'dqn_best_weights.h5')
save_best_weights = SaveBestWeights(filepath=filepath_bw, interval=50, overwrite=True)

filepath_lw = os.path.join(modelos_path, 'dqn_last_weights.h5')
save_last_weights = SaveLastWeights(filepath=filepath_lw, interval=50)

print("########--------------  modelo 3  ------------------###########")

start_time = time.time()

train_scores = dqn_dueling_light_dqn.fit(env, nb_steps=250000, visualize=False, verbose=1, log_interval=10000, 
                       callbacks=[save_best_weights, save_last_weights, FileLogger(log_filename, interval=100)])

end_time = time.time()
training_duration = end_time - start_time

print(f"Entrenamiento completado en {training_duration:.2f} segundos ({training_duration/60:.2f} minutos)")

weights_filename = os.path.join(modelos_path, 'dqn_last_weights.h5')
dqn_dueling_light_dqn.load_weights(weights_filename)
test_scores = dqn_dueling_light_dqn.test(env, nb_episodes=10, visualize=False)

episode_rewards = test_scores.history['episode_reward']
print(int(np.min(episode_rewards)), "~", int(np.max(episode_rewards)), "Mean", np.mean(episode_rewards))

########--------------  modelo 3  ------------------###########
Training for 250000 steps ...
Interval 1 (0 steps performed)


d:\anaconda3\envs\gym4\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


  720/10000 [=>............................] - ETA: 54s - reward: 0.0167
 ##### Pesos guardados en POC3\modelos\dqn_best_weights.h5 recompensa: 12.0

 ##### Pesos guardados en POC3\modelos\dqn_last_weights.h5 al final del episodio 0
10000/10000 [==============================] - 99s 7ms/step - reward: 0.0165
12 episodes - episode_reward: 12.667 [6.000, 21.000] - ale.lives: 2.063

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 66s 7ms/step - reward: 0.0175
11 episodes - episode_reward: 16.455 [9.000, 28.000] - ale.lives: 2.055

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 63s 6ms/step - reward: 0.0179
12 episodes - episode_reward: 14.500 [9.000, 23.000] - ale.lives: 1.941

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 60s 6ms/step - reward: 0.0159
14 episodes - episode_reward: 11.786 [5.000, 21.000] - ale.lives: 2.066

Interval 5 (40000 steps performed)
 1374/10000 [===>..........

In [20]:
path = 'POC4'

# Define subdirectorios
videos_path = os.path.join(path, 'videos')
modelos_path = os.path.join(path, 'modelos')

# Crear directorios si no existen
os.makedirs(videos_path, exist_ok=True)
os.makedirs(modelos_path, exist_ok=True)

log_filename = os.path.join(modelos_path, f'dqn_{env_name}_log.json')

filepath_bw = os.path.join(modelos_path, 'dqn_best_weights.h5')
save_best_weights = SaveBestWeights(filepath=filepath_bw, interval=50, overwrite=True)

filepath_lw = os.path.join(modelos_path, 'dqn_last_weights.h5')
save_last_weights = SaveLastWeights(filepath=filepath_lw, interval=50)

print("########--------------  modelo 4  ------------------###########")

start_time = time.time()

train_scores = dqn_like_dqn.fit(env, nb_steps=250000, visualize=False, verbose=1, log_interval=10000, 
                       callbacks=[save_best_weights, save_last_weights, FileLogger(log_filename, interval=100)])

end_time = time.time()
training_duration = end_time - start_time

print(f"Entrenamiento completado en {training_duration:.2f} segundos ({training_duration/60:.2f} minutos)")

weights_filename = os.path.join(modelos_path, 'dqn_last_weights.h5')
dqn_like_dqn.load_weights(weights_filename)
test_scores = dqn_like_dqn.test(env, nb_episodes=10, visualize=False)

episode_rewards = test_scores.history['episode_reward']
print(int(np.min(episode_rewards)), "~", int(np.max(episode_rewards)), "Mean", np.mean(episode_rewards))

########--------------  modelo 4  ------------------###########
Training for 250000 steps ...
Interval 1 (0 steps performed)
 1074/10000 [==>...........................] - ETA: 1:06 - reward: 0.0102
 ##### Pesos guardados en POC4\modelos\dqn_best_weights.h5 recompensa: 11.0

 ##### Pesos guardados en POC4\modelos\dqn_last_weights.h5 al final del episodio 0
10000/10000 [==============================] - 97s 7ms/step - reward: 0.0133
14 episodes - episode_reward: 9.000 [4.000, 18.000] - ale.lives: 2.171

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 65s 6ms/step - reward: 0.0139
14 episodes - episode_reward: 9.571 [4.000, 23.000] - ale.lives: 2.170

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 66s 7ms/step - reward: 0.0123
14 episodes - episode_reward: 9.500 [1.000, 16.000] - ale.lives: 2.162

Interval 4 (30000 steps performed)
 5614/10000 [===============>..............] - ETA: 29s - reward: 0.0150
 ##### Pesos gua

Resumen de lo obtenido para número de iteraciones = 250.000 


| Model ID | Nombre del Modelo | Recompensas Totales | Recompensa Media | Duración prueba |
|----------|--------------------|----------------------|-------------------|-------------------|
| Model 1  | create_dqn_model         | 2 ~ 26                | 14.6               | 113 m 21 seg 
| Model 2  | create_separable_light_dqn       | 6 ~ 10                | 7.1               | 167 m 18 seg
| Model 3  | create_dueling_light_dqn | 18 ~ 18     | 18               | 43 m
| Model 4  | create_mobile_like_dqn | 18 ~ 20 | 19.6 | 63 m 54 seg

In [ ]:
# # Testing part to calculate the mean reward
# weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
# dqn.load_weights(weights_filename)
# dqn.test(env, nb_episodes=10, visualize=False)

3. Justificación de los parámetros seleccionados y de los resultados obtenidos

---